# Análisis de préstamos de libros en bibliotecas de Madrid con Hadoop

¿Cuáles son los libros más prestados en las bibliotecas públicas de Madrid? En este notebook responderemos a esa pregunta procesando un dataset real de millones de registros de préstamos bibliotecarios. A lo largo del ejercicio, descargaremos los datos, escribiremos un job MapReduce en Python para contar préstamos por título de libro, lo ejecutaremos tanto en local como en un clúster Hadoop, y finalmente visualizaremos los resultados.

**Dataset:** Préstamos de las bibliotecas públicas de Madrid (2025-26) en formato CSV: [https://datos.madrid.es/dataset/212700-0-bibliotecas-prestamos-historico/information](https://datos.madrid.es/dataset/212700-0-bibliotecas-prestamos-historico/information)

## Contenido
1. **Descarga del dataset**
2. **MapReduce local: Mapper y Reducer en Python**
3. **Crear directorios y subir ficheros a HDFS**
4. **Inspeccionar ficheros en HDFS**
5. **Cambiar la replicación de un fichero**
6. **MapReduce con Hadoop Streaming**
7. **Descargar resultados y visualizar con Python**

---

## 1. Descarga del dataset

Descargamos el fichero CSV de préstamos bibliotecarios de Madrid. Este dataset contiene información sobre cada préstamo: código de barras, biblioteca, tipo de material, título, autor, fecha, etc.

In [ ]:
import os

os.makedirs("data", exist_ok=True)
!wget -O data/prestamos_bibliotecas_madrid_2025_26.csv "http://192.168.70.194/prestamos_bibliotecas_madrid_2025_26.csv"
!wc -l data/prestamos_bibliotecas_madrid_2025_26.csv

In [ ]:
!head -n 10 data/prestamos_bibliotecas_madrid_2025_26.csv

El CSV usa `;` como separador. Las columnas son:

| Índice | Campo    | Descripción                          | Tipo/Tamaño        |
|--------|----------|--------------------------------------|---------------------|
| 0      | `prbarc` | Código de barras del ejemplar        | Numérico 10         |
| 1      | `prprsu` | Código de sucursal del préstamo      | Alfanumérico 6      |
| 2      | `prcolp` | Código de tipo de lector             | Alfanumérico 3      |
| 3      | `prcocp` | Código de tipo de ejemplar           | Alfanumérico 3      |
| 4      | `prlebi` | Biblioteca del lector                | Alfanumérico 6      |
| 5      | `prlesu` | Sucursal del lector                  | Alfanumérico 6      |
| 6      | `pradul` | Adulto (1) / Infantil (0)            | Numérico 1          |
| 7      | `prfpre` | Fecha del préstamo (YYYY-MM-DD HH:MM:SS) | Fecha/Hora 24  |
| 8      | `prcocs` | Soporte del ejemplar                 | Alfanumérico 3      |
| 9      | `tititu` | Título (truncado a 60 caracteres)    | Alfanumérico 60     |
| 10     | `tiauto` | Autor                                | Alfanumérico        |

**Códigos de soporte (`prcocs`):**

| Código | Descripción                    | Código | Descripción                    |
|--------|--------------------------------|--------|--------------------------------|
| `LIB`  | Libro                          | `DVV`  | Vídeo (DVD)                    |
| `FOL`  | Folleto                        | `DVR`  | Recurso electrónico (DVD)      |
| `CDA`  | Audio (CD)                     | `VBR`  | Vídeo (Blue-Ray)               |
| `CDR`  | Recurso electrónico (CD)       | `VHV`  | Vídeo (VHS)                    |
| `CAA`  | Audio (casete)                 | `PPE`  | Publicación periódica          |
| `MAP`  | Material cartográfico          | `MFD`  | Microformas                    |
| `MGN`  | Grabados y carteles            | `MUI`  | Música impresa                 |
| `DIR`  | Recurso electrónico (disquete) | `DIA`  | Diapositivas                   |
| `OTA`  | Audio (otros)                  | `OTR`  | Recurso electrónico (otros)    |
| `OTS`  | Otro soporte                   | `OTV`  | Vídeo (otros)                  |
| `VIA`  | Audio (vinilo)                 |        |                                |

## 2. MapReduce local: Mapper y Reducer en Python

Antes de lanzar un trabajo en el clúster Hadoop, es buena práctica probarlo en local. Usamos el patrón **Hadoop Streaming**: los scripts leen de `stdin` y escriben a `stdout`.

### Mapper
Lee cada línea del CSV, filtra líneas vacías y cabeceras, extrae el título y emite `título\t1`:

In [ ]:
%%writefile mapper.py
#!/usr/bin/env python3
import sys
import csv

# CSV con delimitador ';'
# Columnas: prbarc;prprsu;prcolp;prcocp;prlebi;prlesu;pradul;prfpre;prcocs;tititu;tiauto
reader = csv.reader(sys.stdin, delimiter=';')

for row in reader:
    # Saltar líneas vacías o la fila de cabecera
    if not row or 'prbarc' in row[0] or 'prprsu' in row[0]:
        continue

    try:
        # El índice 9 corresponde a tititu (título del libro)
        title = row[9].strip().replace('/', '')

        if title:
            # Emitir título y cuenta de 1, separados por tabulador
            print(f"{title}\t1")

    except (IndexError, ValueError):
        pass

### Reducer
Recibe las líneas ordenadas por clave (título) y suma las ocurrencias:

In [ ]:
%%writefile reducer.py
#!/usr/bin/env python3
import sys

current_title = None
current_count = 0

for line in sys.stdin:
    line = line.strip()

    try:
        title, count_str = line.rsplit('\t', 1)
        count = int(count_str)
    except ValueError:
        continue

    if current_title == title:
        current_count += count
    else:
        if current_title is not None:
            print(f"{current_count}\t{current_title}")

        current_title = title
        current_count = count

if current_title is not None:
    print(f"{current_count}\t{current_title}")

### Prueba local
Probamos la cadena completa en local (sin Hadoop) usando pipes de Unix — exactamente como lo hará Hadoop Streaming internamente:

In [ ]:
# Damos permisos de ejecución a los scripts
!chmod +x mapper.py
!chmod +x reducer.py

# Probamos localmente con las primeras 50 líneas del CSV
!head -n 50 data/prestamos_bibliotecas_madrid_2025_26.csv | ./mapper.py | sort | ./reducer.py | sort -rn | head -10

---

## 3. Crear directorios y subir ficheros a HDFS

In [ ]:
# Crear directorio en HDFS para nuestros datos de bibliotecas
!hadoop fs -mkdir -p libros/input

# Subir el CSV de préstamos a HDFS (-f para sobreescribir si ya existe)
!hadoop fs -put -f data/prestamos_bibliotecas_madrid_2025_26.csv libros/input

# Verificar que el fichero está en HDFS
!hadoop fs -ls -h libros/input

In [ ]:
# Tamaño legible del fichero (-h = human-readable)
!hadoop fs -du -h libros/input/
    
# Espacio en HDFS (human-readable con -h)
!hadoop fs -df -h /

---

## 4. Inspeccionar ficheros en HDFS

Una vez subido el fichero, podemos inspeccionarlo directamente en HDFS sin descargarlo — muy útil para ficheros enormes.

In [ ]:
# Ver las primeras líneas del fichero en HDFS (cabecera + primeros registros)
!hadoop fs -head libros/input/prestamos_bibliotecas_madrid_2025_26.csv

### Información de bloques y replicación

En HDFS, los ficheros se dividen en **bloques** (por defecto 128 MB) que se replican en varios nodos. Podemos ver esta información con `fsck`:

In [ ]:
# Ver información de bloques, replicación y distribución del fichero
!hdfs fsck libros/input/prestamos_bibliotecas_madrid_2025_26.csv -files -blocks -locations

---

## 5. Cambiar la replicación de un fichero

Por defecto HDFS replica cada bloque 4 veces (en nuestro clúster con 4 workers). Podemos cambiar el factor de replicación con `setrep`:

In [ ]:
# Ver la replicación actual (la 2ª columna en ls muestra el factor de replicación)
!hadoop fs -ls libros/input

# Cambiar la replicación a 2
!hadoop fs -setrep 2 libros/input/prestamos_bibliotecas_madrid_2025_26.csv

# Verificar el cambio
!hadoop fs -ls libros/input

In [ ]:
# Restaurar la replicación a 4
!hadoop fs -setrep 4 /user/jovyan/libros/input/prestamos_bibliotecas_madrid_2025_26.csv

---

## 6. MapReduce con Hadoop Streaming

Ahora vamos a ejecutar nuestro mapper y reducer **en el clúster Hadoop** usando **Hadoop Streaming**. En lugar de procesar los datos en una sola máquina, Hadoop distribuirá el trabajo entre los 4 workers.

### ¿Cómo funciona?

```
┌──────────┐     ┌──────────────┐     ┌───────────┐     ┌──────────────┐     ┌──────────┐
│  HDFS    │────▶│   Mapper     │────▶│   Sort &  │────▶│   Reducer    │────▶│  HDFS    │
│  Input   │     │  (mapper.py) │     │  Shuffle  │     │ (reducer.py) │     │  Output  │
└──────────┘     └──────────────┘     └───────────┘     └──────────────┘     └──────────┘
```

- **Input:** el fichero CSV en HDFS se divide en _splits_ (uno por bloque)
- **Map:** cada worker ejecuta `mapper.py` sobre su _split_
- **Shuffle & Sort:** Hadoop agrupa y ordena las claves intermedias (títulos)
- **Reduce:** `reducer.py` suma las ocurrencias de cada título
- **Output:** el resultado se escribe en HDFS

### Ejecutar el trabajo MapReduce

In [ ]:
# Primero eliminamos el directorio de salida si existe (Hadoop no permite sobreescribirlo)
!hadoop fs -rm -r -f libros/output

# Ejecutar MapReduce con Hadoop Streaming
!hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
    -files mapper.py,reducer.py \
    -mapper "python3 mapper.py" \
    -reducer "python3 reducer.py" \
    -input libros/input/prestamos_bibliotecas_madrid_2025_26.csv \
    -output libros/output

### Examinar la salida del MapReduce

La salida se guarda en un directorio de HDFS. Hadoop crea un fichero `part-XXXXX` por cada reducer:

In [ ]:
# Ver los ficheros de salida generados por MapReduce
!hadoop fs -ls libros/output/

In [ ]:
# Top 20 libros más prestados (ordenar numéricamente en orden descendente)
!hadoop fs -cat libros/output/part-00000 | sort -rn 2>/dev/null | head -10

---

## 7. Descargar resultados y visualizar con Python

In [ ]:
# Descargar la salida de MapReduce a local
!hadoop fs -get -f /user/jovyan/libros/output/part-00000 data/resultado_prestamos.tsv

# Verificar
!head -5 data/resultado_prestamos.tsv

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Leer la salida del MapReduce (formato: count\ttitle)
df = pd.read_csv(
    'data/resultado_prestamos.tsv',
    sep='\t',
    header=None,
    names=['prestamos', 'titulo'],
    encoding='utf-8',
    encoding_errors='replace'
)

print(f"Total de títulos únicos: {len(df):,}")
print(f"Total de préstamos: {df['prestamos'].sum():,}")
print()

top = df.nlargest(50, 'prestamos')
print("📚 Top 50 libros más prestados en las bibliotecas de Madrid:")
print(top.to_string(index=False))